# 第3章　教師あり学習のためのデータ前処理

- 澤田 高志
- 清水 秀幸

##### 入力3-1

In [ ]:
!pip install GEOparse

##### 入力3-2

In [ ]:
# パッケージのインポート
%matplotlib inline
import GEOparse
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

##### 入力3-3

In [ ]:
# バージョンの確認
print('numpy: ', np.__version__)
print('pandas: ', pd.__version__)
print('matplotlib: ', matplotlib.__version__)
print('seaborn: ', sns.__version__)
print('GEOparse: ', GEOparse.__version__)

##### 入力3-4

In [ ]:
gse = GEOparse.get_GEO(geo='GSE36376')

##### 入力3-5

In [ ]:
pd.set_option('display.max_columns', 50)  # 30列全てを見えるようにする
gse_table = gse.gpls['GPL10558'].table
gse_table

##### 入力3-6

In [ ]:
# 欠損値ではないデータの数
gse_table.info()

##### 入力3-7

In [ ]:
# gse_table.isnull()では，欠損値はTrueになる
# そこにsum()を加えて，各列ごとの欠損値をカウントする
print(gse_table.isnull().sum())

##### 入力3-8

In [ ]:
# gse_table['Species']が'Homo sapiens'に一致するもののみ抽出
# 'Homo sapiens'ということで'_hs'を末尾に付ける
gse_table_hs = gse_table[gse_table['Species'] == 'Homo sapiens']
gse_table_hs

##### 入力3-9

In [ ]:
# 各列ごとの欠損値をカウントする
print(gse_table_hs.isnull().sum())

##### 入力3-10

In [ ]:
# ['Symbol']列の欠損値をdropnaで除去
# 'Symbol'ということで'_sym'を末尾に付ける
gse_table_hs_sym = gse_table_hs.dropna(subset=['Symbol'])
gse_table_hs_sym

##### 入力3-11

In [ ]:
# 各列ごとの欠損値をカウントする
print(gse_table_hs_sym.isnull().sum())

##### 入力3-12

In [ ]:
# RefSeq_IDの頭文字3文字の内訳をカウント
print(gse_table_hs_sym['RefSeq_ID'].str[:3].value_counts())

##### 入力3-13

In [ ]:
# mRNAのみということで'_NM'を末尾に付ける
gse_table_hs_sym_NM = gse_table_hs_sym[gse_table_hs_sym['RefSeq_ID'].str[:3] == 'NM_']
gse_table_hs_sym_NM

##### 入力3-14

In [ ]:
print(gse_table_hs_sym_NM.isnull().sum())

##### 入力3-15

In [ ]:
symbol_count = gse_table_hs_sym_NM['Symbol'].value_counts()
symbol_dup = symbol_count[symbol_count > 1]
print(symbol_dup)

##### 入力3-16

In [ ]:
gse_phenotype = gse.phenotype_data
gse_phenotype

##### 入力3-17

In [ ]:
# 欠損値は存在しない
print(gse_phenotype.isnull().sum())

##### 入力3-18

In [ ]:
# サンプルIDとそのサンプルが正常か腫瘍かについて
gse_phenotype['characteristics_ch1.0.tissue']

##### 入力3-19

In [ ]:
# 正常サンプルと腫瘍サンプルの数
gse_phenotype['characteristics_ch1.0.tissue'].value_counts()

##### 入力3-20

In [ ]:
# 正常サンプルと腫瘍サンプルのIDのみを抜き出す
normal_ID = gse_phenotype[
    gse_phenotype['characteristics_ch1.0.tissue'] == 'adjacent non-tumor liver'
].index
tumor_ID = gse_phenotype[
    gse_phenotype['characteristics_ch1.0.tissue'] == 'liver tumor'
].index

##### 入力3-21

In [ ]:
# pivot_samplesで発現量をまとめる
pivoted_samples = gse.pivot_samples('VALUE')
pivoted_samples

##### 入力3-22

In [ ]:
# 433列の欠損値の数をカウントする
print(pivoted_samples.isnull().sum())

##### 入力3-23

In [ ]:
# value_countsでpivoted_samples.isnull().sum()にどのような値があったか確認する
print(pivoted_samples.isnull().sum().value_counts())

##### 入力3-24

In [ ]:
# 47323行の欠損値の数をカウントする
pivoted_samples.isnull().sum(axis=1).value_counts()

##### 入力3-25

In [ ]:
# 欠損値を含む行だけ抽出する
pivoted_samples[pivoted_samples.isnull().any(axis=1)]

##### 入力3-26

In [ ]:
# 要約統計量を確認する
pivoted_samples.describe()

##### 入力3-27

In [ ]:
# 全サンプルを図示すると見にくいため，5サンプルのみ
pivoted_samples.iloc[:, 0:5].boxplot()

##### 入力3-28

In [ ]:
# Seabornで描いた箱ひげ図
# Seabornはsnsという形でインポートすることが多い (import seaborn as sns)
sns.boxplot(data=pivoted_samples.iloc[:, 0:5])

##### 入力3-29

In [ ]:
# violinplotは箱ひげ図よりもデータの全体傾向を把握しやすいというメリットがある.
# 「ヴァイオリン」の縁の部分はカーネル密度推定で描かれている
sns.violinplot(data=pivoted_samples.iloc[:, 0:5])

##### 入力3-30

In [ ]:
# boxenplotは大規模なデータセット用の箱ひげ図と言える.
# 四分位しかわからない箱ひげ図と比べ，細かい分位をも理解できる
sns.boxenplot(data=pivoted_samples.iloc[:, 0:5])

##### 入力3-31

In [ ]:
# displotでヒストグラムとカーネル密度推定を同時に見ることができる
# pivoted_samples.iloc[:, 0]は少し見にくいのでpivoted_samples.iloc[:, 1]を見る

current_palette = sns.color_palette(n_colors=24) # seabornでの色の指定を行うため
sns.displot(
    pivoted_samples.iloc[:, 1],
    kde=True,
    facecolor=current_palette[0], # 青色がヒストグラム
    color=current_palette[1],
) # オレンジの線がカーネル密度推定: ヴァイオリンプロットと同じもの

##### 入力3-32

In [ ]:
sns.pairplot(pivoted_samples.iloc[:, 0:5], height=2.5)

##### 入力3-33

In [ ]:
sns.heatmap(pivoted_samples.iloc[:, 0:5].corr(), square=True, annot=True)

##### 入力3-34

In [ ]:
# gse_table_hs_sym_NMから重要な列として
# mRNAのプローブ情報列である'ID'と，重複の除去の必要があるmRNA遺伝子名の列である'Symbol'を抽出する.
col = ['ID', 'Symbol']
gse_selected = gse_table_hs_sym_NM[col]
# 'ID'列はインデックスにしてしまう
gse_selected.index = gse_selected['ID']
gse_selected = pd.DataFrame(gse_selected['Symbol'])
gse_selected

##### 入力3-35

In [ ]:
# gse_selectedのindexに含まれるIDのプローブについて，
# 発現量をまとめる
pivoted_samples_mRNA = pivoted_samples.loc[
    gse_selected.index,
]
pivoted_samples_mRNA

##### 入力3-36

In [ ]:
gse_selected[gse_selected['Symbol'] == 'GAPDH']

##### 入力3-37

In [ ]:
GAPDH_ID = ['ILMN_1343295', 'ILMN_1802252', 'ILMN_2038778']
# mean(axis=1)により，行方向の平均値が求められる.
pivoted_samples_mRNA.loc[
GAPDH_ID,
].mean(axis=1)

##### 入力3-38

In [ ]:
# np.argmaxで，数値が最大となるものの番号(この場合，2)を抜き出す
GAPDH_max_num = np.argmax(
    pivoted_samples_mRNA.loc[
    GAPDH_ID,
    ].mean(axis=1)
)
# 数値が最大となるIDは，0番目の'ILMN_1343295'でも，1番目の'ILMN_1802252'でもなく
# 2番目の'ILMN_2038778'である
GAPDH_ID[GAPDH_max_num]

##### 入力3-39

In [ ]:
# 'Amean'は各行の平均値である。
pivoted_samples_mRNA['Amean'] = pivoted_samples_mRNA.mean(axis=1)

##### 入力3-40

In [ ]:
# pd.merge関数で，Pandasのデータフレームを結合させる.
# left_indexもright_indexもTrueとすることで，互いのインデックスを参照して結合させる.
gse_pivoted_merge = pd.merge(
    gse_selected, pivoted_samples_mRNA, left_index=True, right_index=True
)
# sort_valuesでソートする.ascending = Falseで降順になる.
gse_pivoted_merge_sorted = gse_pivoted_merge.sort_values(['Amean'], ascending=False)
gse_pivoted_merge_sorted

##### 入力3-41

In [ ]:
# 重複している遺伝子については'Amean'が大きいものを選択する.
# 降順にデータが並んでいるところに，drop_duplicates()を，keep = 'first'で使うことで
# 最初に出てきた遺伝子を選択し，以降の重複を除去する.
gse_pivoted_merge_sorted_unique = gse_pivoted_merge_sorted.drop_duplicates(
      ['Symbol'], keep='first'
)

# もはやindexはプローブ(ILMN_)である必要はない.indexを'Symbol'に変更する
gse_pivoted_merge_sorted_unique.index = gse_pivoted_merge_sorted_unique['Symbol']

# 'Symbol'列も'Amean'列ももはや必要ないので除外する.
gse_mRNA_exprs = gse_pivoted_merge_sorted_unique.drop(['Symbol', 'Amean'], axis=1)
gse_mRNA_exprs

##### 入力3-42

In [ ]:
# 欠損値の数は433→155個に
gse_mRNA_exprs.isnull().sum().sum()

##### 入力3-43

In [ ]:
# 欠損値の割合はわずかである
ratio = gse_mRNA_exprs.isnull().sum().sum() / gse_mRNA_exprs.size
print('欠損割合 %f %%' % (ratio * 100))

##### 入力3-44

In [ ]:
# normal_ID, tumor_IDはすでに抽出している
gse_mRNA_exprs_normal = gse_mRNA_exprs[normal_ID]
gse_mRNA_exprs_tumor = gse_mRNA_exprs[tumor_ID]

##### 入力3-45

In [ ]:
# データフレームの平均値を求める.
# axis=1により行方向の平均値を求められる
FC = gse_mRNA_exprs_tumor.mean(axis=1) / gse_mRNA_exprs_normal.mean(axis=1)
# 最小値と最大値を見てみよう
FC.sort_values()

In [ ]:
##### 入力3-46

In [ ]:
# log2FCに変換
log2FC = np.log2(FC)

##### 入力3-47

In [ ]:
# log2FCの絶対値が，0.5を超える，mRNAを抽出する
threshold = 0.5
log2FC[log2FC.abs() > threshold]

##### 入力3-48

In [ ]:
# 浮かび上がってきたmRNAの名前を抽出する
FC_selected_mRNA = list(log2FC[log2FC.abs() > threshold].index)
len(FC_selected_mRNA)

##### 入力3-49

In [ ]:
# 本来，欠損値の処理は非常にデリケートな問題で，欠損値を平均値などで，あるいはテクニカルにベイズなどで
# 補完するか，いっそのこと除去するかは悩ましい問題である.
# 今回は着目している遺伝子に欠損値が含まれていないことを利用し，この問題を回避することができた.
gse_mRNA_exprs_normal_selected = gse_mRNA_exprs_normal.loc[FC_selected_mRNA, :]
gse_mRNA_exprs_normal_selected.isnull().sum().sum()

##### 入力3-50

In [ ]:
gse_mRNA_exprs_tumor_selected = gse_mRNA_exprs_tumor.loc[FC_selected_mRNA, :]
gse_mRNA_exprs_tumor_selected.isnull().sum().sum()

##### 入力3-51

In [ ]:
# submitはラベルと予測結果をまとめたpd.DataFrame
gse_mRNA_exprs_normal_selected.to_csv('GSE36376_normal.csv')
gse_mRNA_exprs_tumor_selected.to_csv('GSE36376_tumor.csv')

# ローカルファイルにダウンロード
from google.colab import files
files.download('GSE36376_normal.csv')
# 2つ目のファイルのダウンロードでは左上にポップアップが出てくる.「許可」を押してほしい.
files.download('GSE36376_tumor.csv')

##### 入力3-52

In [ ]:
# 0行目から1行目までの2行分を抽出して，整然でないデータを見てみる
messy_example = pd.DataFrame(gse_mRNA_exprs_normal_selected.T.iloc[0:2])
# 説明がしやすいため，インデックス情報をいったんデータフレームに入れる
messy_example['Sample_ID'] = messy_example.index
messy_example

##### 入力3-53

In [ ]:
# pd.melt()で整然データに変換
tidy_example = pd.melt(messy_example, id_vars=messy_example.columns[-1])
tidy_example

##### 入力3-54

In [ ]:
# データ全体で整然データを作ってみる
gse_mRNA_exprs_normal_selected_tidy = pd.melt(gse_mRNA_exprs_normal_selected.T)
# 後で図示する際に使うので，'class'列という形で正常サンプルであることを示しておく
# Sample_ID情報は今回はいらない
gse_mRNA_exprs_normal_selected_tidy['class'] = 'Normal sample'
gse_mRNA_exprs_normal_selected_tidy

##### 入力3-55

In [ ]:
gse_mRNA_exprs_tumor_selected_tidy = pd.melt(gse_mRNA_exprs_tumor_selected.T)
# 後で図示する際に使うので，'class'列という形でがんサンプルであることを示しておく
gse_mRNA_exprs_tumor_selected_tidy['class'] = 'Tumor sample'
gse_mRNA_exprs_tumor_selected_tidy

##### 入力3-56

In [ ]:
# pd.concat(axis=0)で正常の整然データとがんの整然データを縦に結合させる
gse_mRNA_exprs_selected_tidy = pd.concat(
    [gse_mRNA_exprs_normal_selected_tidy, gse_mRNA_exprs_tumor_selected_tidy], axis=0
)
gse_mRNA_exprs_selected_tidy

##### 入力3-57

In [ ]:
# figsize=(15, 8)で文字が潰れないようにグラフ全体のサイズの変更をしている
# 各自のパソコンに合わせて値を変更させてほしい
plt.figure(figsize=(15, 8))
# 左右に直接Normal sampleとTumor sampleを並べるviolin plot
# quartileで中央値と四分位数のところに線を引いている
sns.violinplot(
      data=gse_mRNA_exprs_selected_tidy,
      x='Symbol',
      y='value',
      hue='class',
      split=True,
      inner='quartile',
)